# Evaluation
- For retriever
- For generator

## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Titan Embedding 및 LLM 인 Claude-v2.1 모델 로딩

### LLM 로딩 (Claude-v2.1)

In [5]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=False,
    callbacks=[StreamingStdOutCallbackHandler()]
)

llm_text_light= Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-Instant-V1"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=False,
    callbacks=[StreamingStdOutCallbackHandler()]
)

### Embedding 모델 선택

In [7]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [8]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):

    if is_bedrock_embeddings:
        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
                #model_name="Cohere-Embeddings-En"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")

    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [9]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)   

Bedrock Embeddings Model Loaded


## 3. Depoly ReRanker model (if needed)

In [10]:
import json
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [11]:
depoly = False

In [12]:
if depoly:

    try:
        role = sagemaker.get_execution_role()
    except ValueError:
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

    # Hub Model configuration. https://huggingface.co/models
    hub = {
        'HF_MODEL_ID':'BAAI/bge-reranker-large',
        'HF_TASK':'text-classification'
    }

    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        env=hub,
        role=role, 
    )

    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
        initial_instance_count=1, # number of instances
        instance_type='ml.g5.xlarge' # instance type
    )

    print(f'Accept: {predictor.accept}')
    print(f'ContentType: {predictor.content_type}')
    print(f'Endpoint: {predictor.endpoint}')

### Reranker

In [13]:
#endpoint_name = "huggingface-pytorch-inference-2023-11-15-07-53-21-605" # ml.g5.xlarge
endpoint_name = "ko-reranker-serve-2023-12-21-23-45-00-200"

In [14]:
runtime_client = boto3.Session().client('sagemaker-runtime')
print (f'runtime_client: {runtime_client}')

runtime_client: <botocore.client.SageMakerRuntime object at 0x7f1477813310>


## 4. LangChainmOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [15]:
from utils.proc_docs import get_parameter

In [16]:
ssm = boto3.client("ssm", "us-east-1")

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_password',
)

http_auth = (opensearch_user_id, opensearch_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [17]:
index_name = "v17-genai-poc-knox-kor-eval-parent-doc-retriever"

### OpenSearch Client 생성

In [18]:
from utils.opensearch import opensearch_utils

In [19]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 5. Retriever 정의

In [20]:
from utils.rag import OpenSearchHybridSearchRetriever

In [21]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    index_name=index_name,
    llm_text=llm_text, # used in rag_fusion, hyde and reranker(num_tokens)
    llm_emb=llm_emb,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for rank fusion
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.5, .5], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의
    reranker=False, # enable reranker with reranker model
    reranker_endpoint_name=endpoint_name, # endpoint name for reranking model
    #rag_fusion=False, # enable rag_fusion
    #query_augmentation_size=3, # query_augmentation_size in rag_fusion

    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=False,
)

In [22]:
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents("knox")
search_hybrid_result

[Document(page_content='Knox Suite에 대한 액세스 권한을 신청하세요.삼성 Knox 계정을 만들고 Knox 서비스에 액세스하면 Knox Suite에 등록할 수 있습니다. 1.SamsungKnox.com에 로그인하세요. 2.시작 화면이 열리면 시작하기를 클릭합니다.Knox 관리 포털 설정 페이지가 열립니다. 3.Knox 클라우드 솔루션 목록에서 Knox Suite를 선택하고 확인을 클릭합니다.Knox 관리 포털이 열립니다. 4.Knox Suite에 등록하는 마지막 단계는 Knox Manage 등록을 완료하는 것입니다.Knox 관리 포털의 탐색 창에서 Knox 관리를 클릭합니다.열리는 페이지에서 테넌트 ID의 세부 정보를 확인하거나 입력합니다.테넌트 ID로 Knox Manage 계정을 식별하고 직접 URL을 통해 Knox Manage에 로그인할 수 있습니다.조직이 Knox Services에 가입하지 않은 경우 조직의 도메인이 고유한 테넌트 ID를 구성합니다.조직의 도메인이 테넌트 ID 필드를 자동으로 채웠는지 확인하고 등록 완료를 클릭합니다.조직이 이미 Knox Services에 가입한 경우 테넌트를 조직의 다른 테넌트와 구별하기 위해 접두사를 정의해야 합니다.조직의 도메인이 조직 도메인 필드를 자동으로 채웠는지 확인하고 접두사 필드에 접두사를 입력합니다.테넌트 ID는 50자를 초과할 수 없습니다.등록 완료를 클릭합니다.팝업이 열리면 확인 및 완료를 클릭합니다. 5.이제 Knox Suite에 등록되었습니다.Knox Suite 액세스를 신청하면 평가판 Knox Suite 라이선스 키가 자동으로 생성됩니다.Knox 관리 포털의 라이선스 탭에서 확인할 수 있습니다.튜토리얼 진행 3단계 중 2단계를 완료했습니다!이전 다음', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 71, 'title': '녹스 스위트 이용 신청', 'url': 'https://docs.samsungknox.co

## 6. RAG chain 정의

In [23]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

### Prompting
- [TIP] Prompt의 instruction의 경우 한글보다 영어로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [24]:
from utils.rag import prompt_repo

In [25]:
PROMPT = prompt_repo.get_qa(prompt_type="ko_answer_only") # ["answer_only", "answer_with_ref"]

### Update Search Params (Optional)

In [26]:
opensearch_hybrid_retriever.update_search_params(
    k=6,
    minimum_should_match=0,
    filter=[],
    reranker=True,
    reranker_endpoint_name=endpoint_name,
    rag_fusion=False, # enable rag_fusion
    query_augmentation_size=3, # query_augmentation_size in rag_fusion
    hyde=False, # enable hyde
    hyde_query=["web_search"], # query type in hyde 
    parent_document=True,
    llm_text=llm_text, # used in rag_fusion, hyde and reranker(num_tokens)
    verbose=False
)

In [27]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm_text,
    chain_type="stuff",
    retriever=opensearch_hybrid_retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PROMPT,
        "verbose": False,
    },
    verbose=False
)

In [28]:
query = "최종 사용자 입장에서 Knox Capture 기능을 사용하는 데는 어떤 제한 사항이 있나요?"
qa_chain(query)

{'query': '최종 사용자 입장에서 Knox Capture 기능을 사용하는 데는 어떤 제한 사항이 있나요?',
 'result': '\n\n최종 사용자가 Knox Capture 기능을 사용하는 데는 몇 가지 제한 사항이 있습니다.\n\n1. 관리자 모드의 전체 기능을 사용할 수 없습니다. 최종 사용자는 IT 관리자가 정의한 스캔 프로필 설정 내에서만 스캔을 수행할 수 있습니다.\n\n2. 스캔 프로필을 변경하거나 구성 파일을 내보내거나 Knox Capture 라이선스를 업그레이드하여 관리자 모드로 전환할 수 없습니다.\n\n3. 특정 눈부심이나 미세한 조건에서 바코드 감지율이 낮아질 수 있습니다. 사용자는 스캔 전에 바코드의 눈부심을 차단하거나 가려보는 것이 좋습니다.\n\n4. 멀리 떨어진 여러 바코드를 동시에 스캔할 때 문제가 발생할 수 있습니다. 멀티 바코드 스캔 시에는 가능한 가깝게 다가가 스캔하는 것이 좋습니다.',
 'source_documents': [Document(page_content='개요.Knox Capture가 무엇이고 어떻게 작동하는지 알아보고 기본 사항부터 시작해 보세요.녹스 캡처란 무엇인가요?녹스 캡처의 기능을 설명하려면 먼저 *웨지* 스캐너가 무엇인지, 그리고 이 스캐너가 Knox Capture 솔루션과 어떤 관련이 있는지 이해하는 것부터 시작하는 것이 도움이 될 수 있습니다.기존의 하드웨어 기반 바코드 스캔 용어로 웨지 스캐너는 키보드와 컴퓨터 사이에 연결하는 물리적 장치입니다.키보드는 스캐너에 연결되고 스캐너는 컴퓨터에 연결됩니다.컴퓨터의 관점에서 보면 스캐너는 마치 추가 키보드처럼 작동합니다.컴퓨터에서 소프트웨어 프로그램 (예: 인벤토리 스프레드시트 앱) 을 실행하고 웨지 스캐너를 사용하여 바코드를 읽으면 키 입력 형태로 스캐너에서 프로그램으로 데이터가 전송됩니다.프로그램의 관점에서 보면 데이터는 마치 누군가가 키보드를 사용하여 수동으로 입력한 것처럼 보입니다.Knox Capture 작동 방식 비슷한 방식으로 Knox 

## 3.Evaluation
 - [langchain evaluator](https://python.langchain.com/docs/guides/evaluation/string/criteria_eval_chain)

### Load ground truth

In [29]:
import pandas as pd

In [30]:
ground_thruth = pd.read_csv("eval_dataset_v17.csv")

In [31]:
import time
import datetime
from langchain.evaluation import Criteria
from langchain.evaluation import EvaluatorType
from langchain.evaluation import load_evaluator

In [32]:
Criteria.CORRECTNESS

<Criteria.CORRECTNESS: 'correctness'>

In [33]:
list(Criteria)

[<Criteria.CONCISENESS: 'conciseness'>,
 <Criteria.RELEVANCE: 'relevance'>,
 <Criteria.CORRECTNESS: 'correctness'>,
 <Criteria.COHERENCE: 'coherence'>,
 <Criteria.HARMFULNESS: 'harmfulness'>,
 <Criteria.MALICIOUSNESS: 'maliciousness'>,
 <Criteria.HELPFULNESS: 'helpfulness'>,
 <Criteria.CONTROVERSIALITY: 'controversiality'>,
 <Criteria.MISOGYNY: 'misogyny'>,
 <Criteria.CRIMINALITY: 'criminality'>,
 <Criteria.INSENSITIVITY: 'insensitivity'>,
 <Criteria.DEPTH: 'depth'>,
 <Criteria.CREATIVITY: 'creativity'>,
 <Criteria.DETAIL: 'detail'>]

In [34]:
langchain_evaluator = load_evaluator(
    EvaluatorType.LABELED_CRITERIA,
    llm=llm_text,
    criteria=Criteria.CORRECTNESS
)

### Check prompt for evaluation

In [35]:
print(langchain_evaluator.prompt.partial_variables)
print("==")
print(langchain_evaluator.prompt.template)

{'criteria': 'correctness: Is the submission correct, accurate, and factual?'}
==
You are assessing a submitted answer on a given task or input based on a set of criteria. Here is the data:
[BEGIN DATA]
***
[Input]: {input}
***
[Submission]: {output}
***
[Criteria]: {criteria}
***
[Reference]: {reference}
***
[END DATA]
Does the submission meet the Criteria? First, write out in a step by step manner your reasoning about each criterion to be sure that your conclusion is correct. Avoid simply stating the correct answers at the outset. Then print only the single character "Y" or "N" (without quotes or punctuation) on its own line corresponding to the correct answer of whether the submission meets all criteria. At the end, repeat just the letter again by itself on a new line.


### [Optional] Custon evaluator

In [36]:
from langchain.schema.output_parser import StrOutputParser

In [37]:
EVAL_TEMPLATE = """
\n\nHuman: 
Your job is to rate the accuracy of a generated answer given a query and a reference answer.

Here is query: <query>{query}</query>
Here is generated answer: <generated_answer>{generated_answer}</generated_answer>
Here is reference answer: <reference_answer>{reference_answer}</reference_answer>

Then answer the quality of a generated answer as scores starting with "Score:".
Your score has to be between 1 and 5.
You must return your response in a line with only the score.
Do not return answers in any other format.

\n\nAssistant:
"""

prompt_template_eval = PromptTemplate(
    template=EVAL_TEMPLATE, input_variables=["query", "generated_answer", "reference_answer"]
)

question = "What is the default mode of operation for Knox Capture?"
answer="Admin mode is the default mode of operation for Knox Capture. Therefore, a user cannot switch to Admin mode by activating a Knox Capture license, the app is exported in admin mode"
prediction = "The default mode of operation for Knox Capture is Admin mode."


custom_evaluator = prompt_template_eval | llm_text | StrOutputParser()


In [38]:
ground_thruth.head()

,question,answer,doc_id,doc
0,최종 사용자 입장에서 Knox Capture 기능을 사용하는 데는 어떤 제한 사항이...,최종 사용자가 Knox Capture 기능을 사용하는 데는 IT 관리자가 정의한 스...,72e988a4-630c-4ccf-be58-1ef4f558188f,.최종 사용자가 비즈니스 앱을 실행할 때 화면 버튼을 탭하거나 디바이스의 하드웨어 ...
1,앱에서 지원에 액세스하기 위해 누르는 메뉴 아이콘은 무엇인가요?,프로필 목록 레이블 오른쪽에 있는 추가 작업 메뉴 아이콘을 누릅니다.,0e7b9978-0cf9-4be5-9971-1f8742a1cece,문제 해결.앱에서 지원에 액세스하세요. 내에서 지원에 액세스하려면 프로필 목록 레이...
2,스캔 프로필 설정에서 문자 추가 옵션을 활성화하려면 어떤 단계들이 필요한가요?,스캔 프로필 설정에서 문자 추가 옵션을 활성화하려면 아래 단계들이 필요합니다.\n1...,7963a938-f370-4766-ae61-56ea2e8b7344,.스캔 엔진 설정을 변경하려면: 1.바코드 유형을 탭한 다음 아래로 스크롤하여 선택...
3,Knox Capture 구성 데이터에는 어떤 키와 값이 포함되어 있나요?,Knox Capture 구성 데이터에는 smartscan_license 키에 Kno...,323a0ce8-38cc-4c9e-aaca-b5fcbbc37540,.하지만 녹스 캡처 평가판을 사용하는 경우 Knox Suite 라이선스 키도 푸시해...
4,스캐너 연결 화면에서 어떤 아이콘을 탭하여 페어링된 USB 스캐너를 제거할 수 있나요?,스캐너 옆에 있는 톱니바퀴 아이콘을 탭하여 페어링된 USB 스캐너를 제거할 수 있습...,0b4f6a3c-c963-4c14-a905-026c169c0044,. 3.페어링된 USB 스캐너를 제거하려면 스캐너 연결 화면에서 연결된 장치로 이동...


In [39]:
%%time
evaluation = []
parent_document_index = True
for idx, row in enumerate(ground_thruth.itertuples()):
    try:
        question, answer, doc, doc_id = getattr(row, "question"), getattr(row, "answer"), getattr(row, "doc"), getattr(row, "doc_id")

        start = time.time()
        response = qa_chain(question)
        elapsed = time.time() - start

        prediction = response["result"]
        retrieved_docs = {doc.page_content: idx+1 for idx, doc in enumerate(response["source_documents"])}
        contexts = "\n\n".join(retrieved_docs.keys())

        payload = json.dumps(
            {
                "inputs": [
                    {"text": answer, "text_pair": prediction},
                ]
            }
        )

        response = runtime_client.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="application/json",
            Accept="application/json",
            Body=payload
        )
        out = json.loads(response['Body'].read().decode()) ## for json

        eval_result_langchain = langchain_evaluator.evaluate_strings(
            input=question,
            prediction=prediction,
            reference=answer,
        )
        # eval_result_custom = custom_evaluator.invoke(
        #     {
        #         "query": question,
        #         "generated_answer": prediction,
        #         "reference_answer": answer
        #     }
        # )
        
        reranker_sim = out[0]["score"]
        langchain_correctness = eval_result_langchain["score"]
        #custom_correctness = int(eval_result_custom.split("\n")[0].strip().split("Score: ")[1])
        custom_correctness = 0
        
        if not parent_document_index:
            if doc in retrieved_docs:
                true_context_rank = retrieved_docs[doc]
                has_right_context = 1
                mrr = 1/true_context_rank
            else:
                true_context_rank = has_right_context = 0
                mrr = 0
        else:
            for retrieved_doc in retrieved_docs:
                
                if doc in retrieved_doc:
                    true_context_rank = retrieved_docs[retrieved_doc]
                    has_right_context = 1
                    mrr = 1/true_context_rank
                    break
                else:
                    true_context_rank = has_right_context = 0
                    mrr = 0

        total_contexts = len(retrieved_docs)
        latency = datetime.timedelta(seconds=elapsed)

        print(idx)
        print(colored("Question: ", "red"), colored(question, "red"))
        print(colored("GroundTruth: ", "green"), colored(answer, "green"))
        print(colored("Prediction: ", "blue"), colored(prediction, "blue"))
        print(f'ReRanker similarity: {reranker_sim}')
        print(f'Langchain correctness: {langchain_correctness}')
        print(f'Custom correctness: {custom_correctness}')
        print(f'has_right_context: {has_right_context}')
        print(f'true_context_rank: {true_context_rank}/{total_contexts}')
        print(f'mrr: {mrr:.3f}')
        print(f'latency: {latency} secs')
        print(f'prediction length: {len(prediction)}')

        evaluation.append([question, answer, prediction, \
                           has_right_context, true_context_rank, total_contexts, mrr, \
                           reranker_sim, langchain_correctness, custom_correctness, elapsed, contexts, len(prediction)])
    except:
        print ("err", idx)

eval_results = pd.DataFrame(
    evaluation,
    columns=["question", "true_answer", "prediction", \
             "has_right_context", "true_context_rank", "total_contexts", "mrr", \
             "reranker_sim", "langchain_correctness", "custom_correctness", "latency", "contexts", "pred_length"]
)
eval_results.to_csv("eval_results_hybrid_reranker_g3_b1_gas32_final.csv", index=False)
#eval_results.to_pickle("eval_results.pkl")

0
Question:  최종 사용자 입장에서 Knox Capture 기능을 사용하는 데는 어떤 제한 사항이 있나요?
GroundTruth:  최종 사용자가 Knox Capture 기능을 사용하는 데는 IT 관리자가 정의한 스캔 프로필에 따라 기능이 제한됩니다. 관리 모드에서는 앱 기능이 크게 줄어듭니다.
Prediction:  

관리자가 Knox Capture를 EMM/UEM을 사용하여 최종 사용자의 디바이스에 관리형 앱으로 배포한 경우 디바이스는 관리 모드에서 앱을 시작합니다. 

관리 모드에서 Knox Capture 기능이 크게 줄어듭니다. 최종 사용자는 스캔 프로필에서 허용하는 대로 비즈니스 앱을 사용하여 스캔 프로필 설정을 검토하고 바코드 스캔을 수행할 수 있지만 스캔 프로필을 변경하거나 프로필 설정을 내보내거나 Knox Capture 라이선스를 활성화하여 관리자 모드로 전환할 수는 없습니다.

따라서 최종 사용자는 IT 관리자가 정의한 스캔 프로필 내에서만 Knox Capture를 사용할 수 있으며, 프로필 설정을 변경하거나 관리 모드에서 벗어날 수 없습니다.
ReRanker similarity: 0.9999493360519409
Langchain correctness: 1
Custom correctness: 0
has_right_context: 1
true_context_rank: 1/4
mrr: 1.000
latency: 0:00:22.668162 secs
prediction length: 366
1
Question:  앱에서 지원에 액세스하기 위해 누르는 메뉴 아이콘은 무엇인가요?
GroundTruth:  프로필 목록 레이블 오른쪽에 있는 추가 작업 메뉴 아이콘을 누릅니다.
Prediction:  

프로필 목록 레이블 오른쪽에 있는 추가 작업 메뉴 아이콘을 누릅니다.
ReRanker similarity: 0.9999496936798096
Langchain correctness: 1
Custom correctness: 0
has_right_co